In [ ]:
%%HTML

<h1>Artificial Reef Aerial Survey Results</h1>

New York State's Artificial Reefs were developed to increase fisheries habitat. They provide marine fish and other organisms additional opportunities for shelter and foraging and may increase productivity in the areas where they are placed. Anglers visit artificial reef sites to benefit from the increased fishing opportun

In [ ]:
import numpy as np
import pandas as pd
import nbconvert
import datetime
import calendar

In [ ]:
#graphing stuff
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
#%matplotlib inline
from mpl_toolkits.basemap import Basemap

In [ ]:
#get the raw data
def reefBoatCount():
    sourceData = pd.read_csv("C:/Users/maalbino/Documents/project_dev/Reefs/ReefsAerialSurvey_boatcount.csv")
    sourceData['Date'] = pd.to_datetime(sourceData['Date'], format='%m/%d/%Y')
    sourceData.rename(columns={'Number of Boats':'NumberOfBoats','Reef ':'Reef','Boat Type':'BoatType'}, inplace=True)
    return sourceData

### Total boat counts

In [ ]:
boatCount = reefBoatCount()

In [ ]:
#Total number of boats counted over the duration of the survey 1995-2015
totalBoats = int(boatCount['NumberOfBoats'].sum())
#Break down by date, year and month
byDate = boatCount.groupby('Date').sum()
#byYear = boatCount.groupby(boatCount.Date.map(lambda x: x.year)).sum()
#byYear.index.names = ['Year']
#byMonth = boatCount.groupby(boatCount.Date.map(lambda x: x.month)).sum()
#byMonth.index.names = ['Month']
#Break down by reef
byReef = boatCount.groupby('Reef').sum()
#Break down by boat type
byBoatType = boatCount.groupby('BoatType').sum()

In [ ]:
analysisBy = "<p>Analysis performed on 8/24/2014 by MAH for years 1995-2015 using reefSurveyResults.ipynb</p>"
print analysisBy

In [ ]:
#How many flights total
flights = len(byDate)
totalBoatsStr = "<h2>Summary</h2><p>The aerial reef survey observed " + str(totalBoats) + " boats utilizing the reefs on " + str(flights) + " flights between 1995 and 2015.</p>"
print totalBoatsStr

In [ ]:
#How many flights that saw over 100 boats?
moreThan100 = byDate[byDate.NumberOfBoats > 100]
print "Of these " + str(flights) + " flights, " + str(len(moreThan100)) + " registered greater than 100 boats."
topTen = moreThan100.sort_values(['NumberOfBoats'], ascending=False)
print "These are the ten most successful flights:"
topTen.head(n=10)
topTenTbl = topTen.head(n=10)
topTenStr = "<p>Of these " + str(flights) + " flights, " + str(len(moreThan100)) + " registered greater than 100 boats. These are the ten most successful flights:</p>"
topTenTbl

In [ ]:
#Make HTML top ten table
topTenTblhtml = topTenTbl\
    .to_html()\
    #.replace('<table border="1" class="dataframe">','<table class="table table-striped">')  

In [ ]:
#How many boats were counted on any given year
byYear = boatCount.groupby(boatCount.Date.map(lambda x: x.year)).sum()
byYear.index.names = ['Year']
byYear

In [ ]:
#add year and cpue column to df
flightsPerYear = reefBoatCount()
flightsPerYear = flightsPerYear.groupby('Date').sum()
flightsPerYear['Year'] = flightsPerYear.index.map(lambda x: x.year)
byYear['Flights'] = flightsPerYear.groupby('Year').count()
byYear['CPUE'] = byYear['NumberOfBoats'] / byYear['Flights']

In [ ]:
byYear

In [ ]:
#Double check the math
byYear['Flights'].sum()

In [ ]:
#make html table
byYearStr = "<h2>Boat Counts Broken Down By Year</h2>"
byYearhtml = byYear\
    .to_html()\
    #.replace('<table border="1" class="dataframe">','<table class="table table-striped">')
    

In [ ]:
#Graph the annual results, this produces a figure as a separate file
#by year
byYear.plot.bar(title="Aerial Reef Survey Summary By Year")
#plt.show()
plt.savefig('S:/GISWorkGroup/outputs/byYearBar.png')
byYearGraphhtml = '<img src="S:/GISWorkGroup/outputs/byYearBar.png">'

In [ ]:
byMonth = boatCount.groupby(boatCount.Date.map(lambda x: x.month)).sum()
byMonth.index.names = ['Month']
byMonth

In [ ]:
#add year and cpue column to df
flightsPerMonth = reefBoatCount()
flights = flightsPerMonth.groupby('Date').sum()
flights['Month'] = flights.index.map(lambda x: x.month)
#flights
byMonth['Flights'] = flights.groupby('Month').count()
byMonth['CPUE'] = byMonth['NumberOfBoats'] / byMonth['Flights']
byMonth

In [ ]:
#check my math
byMonth['Flights'].sum()

In [ ]:
byMonthStr = "<h2>Boat Counts Broken Down By Month</h2>"
byMonthhtml = byMonth\
    .to_html()\
    #.replace('<table border="1" class="dataframe">','<table class="table table-striped">')

In [ ]:
#by month
byMonth.plot.bar(title="Aerial Reef Survey Summary By Month")
plt.savefig('S:/GISWorkGroup/outputs/byMonthBar.png')
byMonthGraphhtml = '<img src="S:/GISWorkGroup/outputs/byMonthBar.png">'

In [ ]:
#We want to know which reef has had the greatest use over the entirety of the survey
reefs = boatCount.groupby('Reef').sum()
reefs = reefs.sort_values(['NumberOfBoats'], ascending=False)
reefs

In [ ]:
byReefStr = "<h2>Boat Counts Broken Down By Reef</h2>"
byReefhtml = reefs\
    .to_html()

In [ ]:
reefs.plot.pie(y='NumberOfBoats', figsize=(8,8), autopct='%.1f', legend=False)
plt.savefig('S:/GISWorkGroup/outputs/byReefPie.png')
byReefGraphhtml = '<img src="S:/GISWorkGroup/outputs/byReefPie.png">'

## Break down by boat size


In [ ]:
moreBoats = reefBoatCount()
bySize = moreBoats.groupby('BoatType').sum()
bySize['Percentage'] = (bySize['NumberOfBoats'] / totalBoats)*100
bySize

In [ ]:
#Basic Map
reefLocations = pd.read_csv("C:/Users/maalbino/Documents/opendata/Reefs/reefPoints.csv")

## Break down by boat size

In [ ]:
#Create HTML page
htmlStart = '''
<html>
    <body>
        <img src="http://internal.dec.state.ny.us/images/dpaeimages/logodecweb.png" width="400"><br>
        <font face="Arial" color="#2C5234">
        <h1>Artificial Reef Survey Results</h1> </font>
        <font face="Arial">'''

htmlMiddle1 = "\n"+ totalBoatsStr + "\n" +  topTenStr + topTenTblhtml + "<hr>" + byYearStr + byYearhtml + byYearGraphhtml+"<hr>"\
    + byMonthStr + byMonthhtml + byMonthGraphhtml +"<hr>" + byReefStr + byReefhtml + byReefGraphhtml


htmlEnd = '<font face="Arial" size="1">' + analysisBy + "</font></font></body></html>"

htmlString = htmlStart + "<hr>" + htmlMiddle1 + "<hr>" + htmlEnd
f = open('C:/Users/maalbino/Desktop/test1.html', 'w')
f.write(htmlString)
f.close()